<center><h1>Stochastic Search</h1></center>
<center><h2>Objective: Find the Global Optima for the Objective function.</h2></center>

__Stochastic Search__ is an iterative method to find the global optima
of a function in the search space. It uses information about the functional value to generate children points around the parent and gradually moving towards the global optima.

---

## Overview of the Algorithm

1. Define the Objective function to be optimized.
2. Generate a set of points of size equal to __popsize__ in the search space.
3. Calculate the functional values of all the points.
4. Compute the weight values for all the points in the set.
5. Calculate the number of children to be generated for all the population points using the weight values.
6. Define the neighbourhood for all the points and generate the calculated children points for all the population.
7. Generate __nRRI__ points by random sampling the entire search space.
8. Generate __nRLC__ points by random linear recombination of population points.
9. Create a super set by adding the parent points, children points, __nRRI__ points and __nRLC__ points to it.
10. Select the __Best__ popsize number of points from the super set.
11. Redefine the population with the best select points.
12. Do the next iteration with the new set of points.

### Demonstration of Stochastic Search

__OBJECTIVE FUNCTION__

maximize \begin{multline}
\label{equ:function}
z = 1.7*exp\bigg[-\bigg\{\dfrac{{(x-3)}^2}{10}+\dfrac{{(y-3)}^2}{10}\bigg\}\bigg]+
exp\bigg[-\bigg\{\dfrac{{(x+5)}^2}{8}+\dfrac{{(y+5)}^2}{8}\bigg\}\bigg] +\\ 2*exp\bigg[-\bigg\{\dfrac{{x}^2}{4}+\dfrac{{y}^2}{5}\bigg\}\bigg] +
1.5*exp\bigg[-\bigg\{\dfrac{{(x-4)}^2}{18}+\dfrac{{(y+4)}^2}{16}\bigg\}\bigg]+\\
1.2*exp\bigg[-\bigg\{\dfrac{{(x+4)}^2}{18}+\dfrac{{(y-4)}^2}{16}\bigg\}\bigg]
\end{multline}

In [1]:
%matplotlib

Using matplotlib backend: <object object at 0x000001DD5889D2B0>


In [2]:
import numpy as np
import matplotlib.pyplot as plt
from math import exp

In [3]:
def fun(x,y):  # Objective Function
    z=1.7*exp(-1*(((x-3)**2)/10+((y-3)**2)/10))+exp(-1*(((x+5)**2)/8+((y+5)**2)/8))+2*exp(-1*(((x)**2)/4+((y)**2)/5))+1.5*exp(-1*(((x-4)**2)/18+(y+4)**2/16))+1.2*exp(-1*(((x+4)**2)/18+((y-4)**2)/16))
    return(z)

In [4]:
def initialise(xmin,xmax,ymin,ymax,popsize):
    x = np.random.uniform(low=xmin, high=xmax, size=(popsize,1))  ## Generating the initial population
    y = np.random.uniform(low=ymin, high=ymax, size=(popsize,1))
    return (np.hstack((x,y)))

In [5]:
def NRRI(xmin,xmax,ymin,ymax,nrri):
    x = np.random.uniform(low=xmin,high=xmax,size=(nrri,1)) # Generating the nRRI points
    y = np.random.uniform(low=ymin,high=ymax,size=(nrri,1))
    return (np.hstack((x,y)))

In [6]:
def NRLC(data,nrlc):
    data_nrlc = np.empty((nrlc,2))
    for i in range(nrlc):    # Generating the nRLC points
        r1=np.random.randint(len(data)) # Randomly selecting points from parent to generate
        r2=np.random.randint(len(data)) # linear combination points
        eta = np.random.rand()
        data_nrlc[i,0] = data[r1,0]*eta+(1-eta)*data[r2,0]
        data_nrlc[i,1] = data[r1,1]*eta+(1-eta)*data[r2,1]
    return data_nrlc

In [7]:
def stochasticsearch(xmin,xmax,ymin,ymax,popsize,radius,nrri,nrlc,maxitr):

    data = initialise(xmin,xmax,ymin,ymax,popsize)  # Initialise the Population

    for i in range(maxitr):  ## Repeating this for maxitr number of times
        data_new = data
        f = []
        for x,y in zip(data[:,0],data[:,1]):  # calculating the functional values for the population
            f.append(fun(x,y))
        f = np.array(f)
        fmin = min(f)

        positive = f - fmin                 # Substracting the min. value of z to make z-value Positive
        W = positive/sum(positive)          # Calculating the weights values
        n = np.round(W*popsize).astype(int) # Calculating the number of children points for each parent point

        for i in range(popsize):
            data_child = np.empty((n[i],2))
            for j in range(n[i]):                           # Generating Children Points around the parent points
                theta = np.random.uniform(low=0,high=360)  # Define the neighbourhood for each parent point
                data_child[j,0] = data[i,0] + radius*np.cos(theta)
                data_child[j,1] = data[i,1] + radius*np.sin(theta)

            data_new = np.append(data_new,data_child,axis=0)   # Appending the childern points in the super list

        data_nrri = NRRI(xmin,xmax,ymin,ymax,nrri)  # function call to generate nrri points

        data_new = np.append(data_new,data_nrri,axis=0)

        data_nrlc = NRLC(data,nrlc)    # function call to generate nrlc points

        data_new = np.append(data_new,data_nrlc,axis=0)

        z = []
        for x,y in zip(data_new[:,0],data_new[:,1]):  # calculating the functional values for the population
            z.append(fun(x,y))
        sort_index = np.argsort(z)[::-1]
        data_new = data_new[sort_index]

        ### For plotting the contour and the scattering the population

        plt.figure(figsize=(8, 7), dpi=80)
        xx = np.linspace(-10,10,100)
        yy = np.linspace(-10,10,100)
        zz = np.empty((100,100))
        XX,YY = np.meshgrid(xx,yy)
        for i in range(100):
            for j in range(100):
                zz[i][j] = fun(XX[i,j],YY[i,j])
        plt.contour(XX,YY,zz)
        plt.scatter(np.array(data[:,0]),np.array(data[:,1]),s=10,color='red')
        plt.show()
        plt.pause(1)
        plt.clf()

        data = data_new[0:popsize]  # Overwriting the previous population
    plt.close('all')
    return data

In [8]:
data = stochasticsearch(-10,10,-10,10,100,0.5,50,50,15)
best = data[0]
z = fun(data[0,0],data[0,1])
print("x,y:",data[0,0],data[0,1])
print("f(x,y)= z =",z)

x,y: 0.20895339085683134 0.21652881508498048
f(x,y)= z = 2.7259306750372674
